<a href="https://colab.research.google.com/github/blanc88/deepLearningChallenge/blob/main/AlphabetSoupCharity_Optimization.h5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [37]:
application_df["SPECIAL_CONSIDERATIONS"].value_counts()

SPECIAL_CONSIDERATIONS
N    34272
Y       27
Name: count, dtype: int64

In [38]:
application_df["STATUS"].value_counts()

STATUS
1    34294
0        5
Name: count, dtype: int64

In [39]:
application_df = application_df.drop(["EIN", "SPECIAL_CONSIDERATIONS", "STATUS"], axis= 1)
application_df

,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,0,5000,1
1,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1-9999,108590,1
2,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,0,5000,0
3,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,10000-24999,6692,1
4,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,100000-499999,142590,1
...,...,...,...,...,...,...,...,...,...
34294,THE LIONS CLUB OF HONOLULU KAMEHAMEHA,T4,Independent,C1000,ProductDev,Association,0,5000,0
34295,INTERNATIONAL ASSOCIATION OF LIONS CLUBS,T4,CompanySponsored,C3000,ProductDev,Association,0,5000,0
34296,PTA HAWAII CONGRESS,T3,CompanySponsored,C2000,Preservation,Association,0,5000,0
34297,AMERICAN FEDERATION OF GOVERNMENT EMPLOYEES LO...,T5,Independent,C3000,ProductDev,Association,0,5000,1


In [40]:
application_df.nunique()

NAME                19568
APPLICATION_TYPE       17
AFFILIATION             6
CLASSIFICATION         71
USE_CASE                5
ORGANIZATION            4
INCOME_AMT              9
ASK_AMT              8747
IS_SUCCESSFUL           2
dtype: int64

In [41]:
total_names = application_df["NAME"].value_counts()
total_names[total_names > 5]

NAME
PARENT BOOSTER USA INC                             1260
TOPS CLUB INC                                       765
UNITED STATES BOWLING CONGRESS INC                  700
WASHINGTON STATE UNIVERSITY                         492
AMATEUR ATHLETIC UNION OF THE UNITED STATES INC     408
                                                   ... 
OLD OAK CLIFF CONSERVATION LEAGUE INC                 6
AMERICAN NEPHROLOGY NURSES ASSOCIATION                6
HUMBLE ISD EDUCATIONAL SUPPORT GROUPS INC             6
PROFESSIONAL LOADMASTER ASSOCIATION                   6
CBMC INC                                              6
Name: count, Length: 354, dtype: int64

In [42]:
replace_names = list(total_names[total_names <= 5].index)

for app in replace_names:
  application_df["NAME"] = application_df["NAME"].replace(app, "Other")

application_df["NAME"].value_counts()

NAME
Other                                           20043
PARENT BOOSTER USA INC                           1260
TOPS CLUB INC                                     765
UNITED STATES BOWLING CONGRESS INC                700
WASHINGTON STATE UNIVERSITY                       492
                                                ...  
HABITAT FOR HUMANITY INTERNATIONAL                  6
DAMAGE PREVENTION COUNCIL OF TEXAS                  6
FLEET RESERVE ASSOCIATION                           6
HUGH OBRIAN YOUTH LEADERSHIP                        6
INTERNATIONAL CONGRESS OF CHURCHES MINISTERS        6
Name: count, Length: 355, dtype: int64

In [43]:
application_totals = application_df["APPLICATION_TYPE"].value_counts()
application_totals

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [44]:
app_type_replace = list(application_totals[application_totals < 500].index)

for app in app_type_replace:
  application_df["APPLICATION_TYPE"] = application_df["APPLICATION_TYPE"].replace(app, "Other")

application_df["APPLICATION_TYPE"].value_counts()


APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [45]:
class_totals = application_df["CLASSIFICATION"].value_counts()
class_totals

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [46]:
class_replace = list(class_totals[class_totals < 1000].index)

for cls in class_replace:
  application_df["CLASSIFICATION"] = application_df["CLASSIFICATION"].replace(cls, "Other")

application_df['CLASSIFICATION'].value_counts()


CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [47]:
app_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()
app_cat

['NAME',
 'APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT']

In [48]:
application_dummies = pd.get_dummies(application_df)
application_dummies.head(5)

,ASK_AMT,IS_SUCCESSFUL,NAME_AACE INTERNATIONAL,NAME_ACE MENTOR PROGRAM OF AMERICA INC,NAME_AFRICAN-AMERICAN POSTAL LEAGUE UNITED FOR SUCCESS A-PLUS,NAME_AIR FORCE ASSOCIATION,NAME_ALABAMA FEDERATION OF WOMENS CLUBS,NAME_ALABAMA TREASURE FOREST ASSOCIATION,NAME_ALBANY STATE UNIVERSITY NATIONAL ALUMNI ASSOCIATION,NAME_ALPHA PHI OMEGA,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,5000,1,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
1,108590,1,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False
2,5000,0,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
3,6692,1,False,False,False,False,False,False,False,False,...,True,False,False,True,False,False,False,False,False,False
4,142590,1,False,False,False,False,False,False,False,False,...,True,False,False,False,True,False,False,False,False,False


In [49]:
x = application_dummies.drop(["IS_SUCCESSFUL"], axis = "columns").values
y = application_dummies["IS_SUCCESSFUL"].values

X_train, X_test, y_train, y_test = train_test_split(x,y)

In [50]:
scaler = StandardScaler()

X_scaler = scaler.fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


In [51]:
input_number = len(X_train[0])
hidden_nodes_layer1 = 100
hidden_nodes_layer2 = 30
hidden_nodes_layer3 = 10

nn = tf.keras.models.Sequential()

# First layer
nn.add(tf.keras.layers.Dense(units = hidden_nodes_layer1, input_dim = input_number, activation = 'relu'))

#Second layer
nn.add(tf.keras.layers.Dense(units = hidden_nodes_layer2, activation = "sigmoid"))

#Third layer
nn.add(tf.keras.layers.Dense(units = hidden_nodes_layer3, activation = "sigmoid"))

#Output layer
nn.add(tf.keras.layers.Dense(units = 1, activation = "sigmoid"))

nn.summary()



Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 100)               39600     
                                                                 
 dense_8 (Dense)             (None, 30)                3030      
                                                                 
 dense_9 (Dense)             (None, 10)                310       
                                                                 
 dense_10 (Dense)            (None, 1)                 11        
                                                                 
Total params: 42951 (167.78 KB)
Trainable params: 42951 (167.78 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [52]:
nn.compile(loss= "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])


In [53]:
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 3s 2ms/step - loss: 0.5118 - accuracy: 0.7591
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4472 - accuracy: 0.7929
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4372 - accuracy: 0.7952
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4334 - accuracy: 0.7975
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4305 - accuracy: 0.7965
Epoch 6/100
804/804 [==============================] - 3s 4ms/step - loss: 0.4278 - accuracy: 0.7983
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4254 - accuracy: 0.7995
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4241 - accuracy: 0.7991
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4229 - accuracy: 0.8005
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4221 - accura

In [54]:
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)

print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.4480 - accuracy: 0.7878 - 525ms/epoch - 2ms/step
Loss: 0.4479885399341583, Accuracy: 0.7877551317214966


In [55]:
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

In [56]:
rf_model = RandomForestClassifier(n_estimators = 128, random_state = 78)

rf_model = rf_model.fit(X_train_scaled, y_train)

y_pred = rf_model.predict(X_test_scaled)
print (f"Random forest model accuracy: {accuracy_score(y_test,y_pred)*100:.3f}")

Random forest model accuracy: 77.353


In [57]:
nn.save("AlphabetSoupCharity_Optimization.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
